In [1]:
import pandas as pd
import sqlalchemy as db
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker

In [2]:
user = 'root'
password = 'MyNewPass'
host = 'localhost'
database = 'Advance_Python'
port = 3306

In [13]:
with db.create_engine(f'mysql+pymysql://{user}:{password}@{host}').connect() as connection:
    connection.execute("DROP DATABASE IF EXISTS Advance_Python")
    connection.execute(f'CREATE DATABASE {database}')
    connection.execute("USE Advance_Python")
    
engine = db.create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
Base = declarative_base()

In [14]:
class Person(Base):
 
    __tablename__ = 'person'
    id = db.Column(db.String(8),primary_key=True)
    name  = db.Column(db.String(32))
    
    cast_relationship = relationship("Cast", back_populates="person_relationship")
    crew_relationship = relationship("Crew", back_populates="person_relationship")

In [15]:
class Movie(Base):
 
    __tablename__ = 'movie'

    id = db.Column(db.String(8),primary_key=True)
    title  = db.Column(db.String(128),nullable=False)
    year = db.Column(db.Integer,nullable=True)
    runtime = db.Column(db.Integer,nullable=True)
    parental_guide = db.Column(db.String(8),nullable=True)
    gross_US_Canada = db.Column(db.Integer,nullable=True)
    
    cast_relationship = relationship("Cast", back_populates="movie_relationship")
    crew_relationship = relationship("Crew", back_populates="movie_relationship")
    genre_relationship = relationship("Genre_movie", back_populates="movie_relationship")
    storyline_relationship = relationship("Storyline", back_populates="movie_relationship")



In [16]:
class Genre_movie(Base):
 
    __tablename__ = 'genre_movie'
    
    id = db.Column(db.Integer, autoincrement=True, nullable=False, primary_key=True)
    movie_id = db.Column(db.String(8),ForeignKey('movie.id'),nullable=False)
    genre = db.Column(db.String(16),nullable=False)
    movie_relationship = relationship("Movie", back_populates="genre_relationship")

In [17]:
class Storyline(Base):
 
    __tablename__ = 'storyline'
    __table_args__ = {'extend_existing': True}
    
    movie_id = db.Column(db.String(8),ForeignKey('movie.id'),primary_key=True,nullable=False)
    content = db.Column(db.Text, nullable=False)
    movie_relationship = relationship("Movie", back_populates='storyline_relationship')

In [18]:
class Cast(Base):
 
    __tablename__ = 'cast'
    __table_args__ = {'extend_existing': True}
    
    id = db.Column(db.Integer, autoincrement=True, nullable=False, primary_key=True)
    movie_id = db.Column(db.String(8),ForeignKey('movie.id'),nullable=False)
    person_id = db.Column(db.String(16),ForeignKey('person.id'), nullable=False)
    
    movie_relationship = relationship('Movie', back_populates='cast_relationship')
    person_relationship = relationship('Person', back_populates='cast_relationship')

In [19]:
class Crew(Base):
 
    __tablename__ = 'crew'
    
    id = db.Column(db.Integer, autoincrement=True, nullable=False, primary_key=True)
    movie_id = db.Column(db.String(8),ForeignKey('movie.id'),nullable=False)
    person_id = db.Column(db.String(16),ForeignKey('person.id'), nullable=False)
    role = db.Column(db.String(8),nullable=False)


    movie_relationship = relationship('Movie', back_populates='crew_relationship')
    person_relationship = relationship('Person', back_populates='crew_relationship')
    

In [20]:
Base.metadata.create_all(engine)
Session = sessionmaker(bind = engine)

In [21]:
movie_df = pd.read_csv('./Fixed_Data/movies.csv',dtype=object)
storyline_df = pd.read_csv('./Fixed_Data/storyline.csv',dtype=object)
genre_movie_df = pd.read_csv('./Fixed_Data/genre_movie.csv',dtype=object)
person_df = pd.read_csv('./Fixed_Data/person.csv',dtype=object)
crew_df = pd.read_csv('./Fixed_Data/crew.csv',dtype=object)
cast_df = pd.read_csv('./Fixed_Data/cast.csv',dtype=object)

In [22]:
session = Session()
for ind in person_df.index:
    session.add(Person(id = person_df['id'][ind], name = person_df['Name'][ind]))
    
session.commit()
for ind in movie_df.index:
    session.add(Movie(id = movie_df['id'][ind], title = movie_df['Title'][ind],
                year = movie_df['Year'][ind], runtime = movie_df['Runtime'][ind],
                parental_guide = movie_df['Parental_Guide'][ind],
                gross_US_Canada = movie_df['Gross_US_Canda'][ind]))
session.commit()

for ind in storyline_df.index:
    session.add(Storyline(movie_id = storyline_df['Movie_id'][ind], content = storyline_df['Content'][ind]))
session.commit()

for ind in genre_movie_df.index:
    session.add(Genre_movie(movie_id = genre_movie_df['Movie_id'][ind], genre = genre_movie_df['Genre'][ind]))
session.commit()

for ind in cast_df.index:
    session.add(Cast(movie_id = cast_df['Movie_id'][ind], person_id = cast_df['Person_id'][ind]))
session.commit()

for ind in crew_df.index:
    session.add(Crew(movie_id = crew_df['Movie_id'][ind],
                     person_id = crew_df['Person_id'][ind],
                     role = crew_df['Role'][ind]))
session.commit()